In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from kneed import KneeLocator

import string
import re

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
lemmatizer = nltk.stem.WordNetLemmatizer()
nltk.download('averaged_perceptron_tagger')


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ekaterinamazur/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/ekaterinamazur/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [7]:
#read the pkl file for movies
df = pd.read_pickle('./data/full_movies.pkl')
#read the pkl file for shows
df_2 = pd.read_pickle('./data/full_shows.pkl')

In [8]:
#read json file activity for movies
# activity_movie = pd.read_json('./movie_activities.json')
# activity_shows = pd.read_json('./show_activities.json')
activities = pd.read_json('./data/activities.json')

In [9]:
activities.head()
activities.groupby('user_id').count()

,content_id,activity,datetime
user_id,,,
0,426,426,426
1,406,406,406
2,386,386,386
3,414,414,414
4,420,420,420
5,233,233,233
6,219,219,219
7,227,227,227
8,227,227,227


## TF-IDF + kmeans clustering

In [10]:
stop_words = stopwords.words('english')
def clean_text(x):
    x = x.lower()
    x = x.translate(str.maketrans('', '', string.punctuation))
    # tokenize
    x = nltk.word_tokenize(x)
    x = [token for token in x if token not in stop_words]
    # lemmatization and pass string back
    x = ' '.join([lemmatizer.lemmatize(w) for w in x])
    return x

In [11]:
df['features'] = df['description'].apply(lambda x: clean_text(x))
df_2['features'] = df_2['description'].apply(lambda x: clean_text(x))

In [12]:
#tf-idf and kmeans for all movies
# Convert the text data to a matrix of TF-IDF features
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf = tfidf_vectorizer.fit_transform(df['features'])


inertia = []
#calculate the optimal number of clusters
K = range(1, 100)
for k in K:
    kmeanModel = KMeans(n_clusters=k,random_state=0)
    kmeanModel.fit(tfidf)
    inertia.append(kmeanModel.inertia_)


kn = KneeLocator(K, inertia, curve='convex', direction='decreasing')
print(kn.knee)


None


/Users/ekaterinamazur/anaconda3/lib/python3.9/site-packages/kneed/knee_locator.py:311: UserWarning: No knee/elbow found
  warnings.warn("No knee/elbow found")


In [13]:
#tf-idf and kmeans for all tv-shows
# Convert the text data to a matrix of TF-IDF features
tfidf_vectorizer_shows = TfidfVectorizer(stop_words='english')
tfidf_shows = tfidf_vectorizer_shows.fit_transform(df_2['features'])


inertia = []
#calculate the optimal number of clusters
K = range(1, 20)
for k in K:
    kmeanModel = KMeans(n_clusters=k,random_state=0)
    kmeanModel.fit(tfidf_shows)
    inertia.append(kmeanModel.inertia_)


kn = KneeLocator(K, inertia, curve='convex', direction='decreasing')
print(kn.knee)

8


In [14]:
# Cluster the documents using KMeans
num_clusters = 13
km = KMeans(n_clusters=num_clusters)
km.fit(tfidf)

# Assign cluster labels to each document
clusters = km.labels_.tolist()
df['cluster'] = clusters
df.head()

,episode_title,id,category,title,series,episode_name,description,description2,tags,image,directors_actors,tags2,publication_date,rating,duration_sec,season,episode,features,cluster
0,The Way We Were,0,Movies,The Way We Were,NaN,The Way We Were,A love story that begins with the attraction o...,A love story that begins with the attraction o...,"[ABC TV, ABC TV Plus, DRAMA, MOVIES]",https://cdn.iview.abc.net.au/thumbs/i/zy/ZY933...,[['Director Sydney Pollack']],"[abc1, abc2, drama, classic, film, feature-len...",2021-06-25 07:00:00,PG,6802.0,NaN,,love story begin attraction opposite katie hub...,3
1,Starman,1,Movies,Starman,NaN,Starman,An alien stranded on Earth clones himself into...,An alien stranded on Earth clones himself into...,"[ABC TV, ABC TV Plus, DRAMA, MOVIES]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW301...,[['Director John Carpenter']],"[abc1, abc2, drama, sci-fi, fantasy, romance, ...",2021-06-25 07:00:00,PG,6612.0,NaN,,alien stranded earth clone form young woman de...,8
2,Mr Deeds Goes To Town,2,Movies,Mr Deeds Goes To Town,NaN,Mr Deeds Goes To Town,A small-town poet inherits a vast fortune and ...,A small-town poet inherits a vast fortune and ...,"[ABC TV, ABC TV Plus, DRAMA, MOVIES]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW302...,"[['Cast Gary Cooper', 'Jean Arthur', 'Douglas ...","[abc1, abc2, drama, comedy-drama, film, featur...",2021-06-25 07:00:00,G,6649.0,NaN,,smalltown poet inherits vast fortune find new ...,0
3,On The Road,3,Movies,On The Road,NaN,On The Road,Aspiring writer Sal Paradise has his world roc...,Aspiring writer Sal Paradise has his world roc...,"[ABC TV, ABC TV Plus, DRAMA, MOVIES]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW291...,[['Director Walter Salles']],"[abc1, abc2, usa, drama, adaptation, adventure...",2021-06-01 07:00:00,MA,8014.0,NaN,,aspiring writer sal paradise world rocked foll...,1
4,The Boys Are Back,4,Movies,The Boys Are Back,NaN,The Boys Are Back,"Journalist Joe Warr and his two sons, from dif...","Journalist Joe Warr and his two sons, from dif...","[ABC TV, ABC TV Plus, DRAMA, MOVIES]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW238...,[['Director Scott Hicks']],"[abc1, abc2, drama, comedy-drama, film, single...",2021-06-11 07:00:00,M,5950.0,NaN,,journalist joe warr two son different marriage...,10


In [15]:
# Cluster the documents using KMeans
num_clusters_shows = 8
km_shows = KMeans(n_clusters=num_clusters_shows)
km_shows.fit(tfidf_shows)

# Assign cluster labels to each document
clusters_shows = km_shows.labels_.tolist()
df_2['cluster'] = clusters_shows
df_2.reset_index(inplace=True)
df_2.head()

,index,id,title,category,description,description2,directors_actors,publication_date,rating,duration_sec,tags,tags2,image,features,cluster
0,0,10000,199 Little Heroes,Education,The journey to school is a very special type o...,"[Enjo lives in Quinten, a forest glade in the ...",[[]],2021-06-01 06:20:00,G,"[311.0, 311.0, 312.0, 311.0, 311.0, 320.0, 311...","[ABC ME, EDUCATION]","[primary-humanities, education, abc3, refugee-...",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW073...,journey school special type life experience sh...,7
1,1,10001,7.30,News,Leigh Sales presents Australia's leading night...,[The devastation caused by the recent floods i...,[[]],2022-03-14 20:00:00,None,"[1906.0, 1894.0, 1879.0, 1935.0, 1891.0, 1822....","[ABC TV, ABC NEWS]","[lismore, news, childcare, flooding, abc-faves...",https://cdn.iview.abc.net.au/thumbs/i/nc/NC220...,leigh sale present australia leading nightly p...,1
2,2,10002,7.30 Mark Humphries Satire,News,Satirist Mark Humphries brings his unique pers...,[Satirist Mark Humphries goes inside the Healt...,[[]],2021-05-11 14:00:00,None,"[145.0, 110.0, 118.0, 136.0, 202.0, 115.0, 241...",[ABC NEWS],"[mark, parachuting, humphries, candidates, sus...",https://cdn.iview.abc.net.au/thumbs/i/nn/NN211...,satirist mark humphries brings unique perspect...,2
3,3,10003,7.30: The Interviews,News,Revisit some of 7.30's most compelling convers...,[Musician Dave Grohl sits down with 7.30's Lei...,[[]],2022-03-08 20:00:00,None,[1030.0],[ABC NEWS],"[news24, news]",https://cdn.iview.abc.net.au/thumbs/i/ns/NS224...,revisit 730s compelling conversation australia...,5
4,4,10004,72 Cutest Animals,Family,The animal kingdom continually astounds us wit...,[Cute is a notion that can mean many things to...,[[]],2022-03-05 15:03:06,G,"[1616.0, 1616.0, 1620.0, 1620.0, 1614.0, 1620....","[ABC ME, FAMILY]","[family-viewing, family-viewing-abcme, abc3]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW243...,animal kingdom continually astounds u showcase...,0


In [16]:
australian = 'Australia|australia|australian|Australian|Aboriginal|Indigenous|First People'
diverse = 'queer|lbgtqi|diversity|inclusion|disability|aboriginal|race|racism|indigenous|equal-opportunit'

In [17]:
expat_library = df[df['tags2'].str.contains(australian) | df['description'].str.contains(australian) | df['description2'].str.contains(australian) | df['title'].str.contains(australian)
]
expat_library_shows= df_2[df_2['tags2'].str.contains(australian) | df_2['description'].str.contains(australian) | df_2['description2'].str.contains(australian) | df_2['title'].str.contains(australian)
]
diversity_library = df[df['tags2'].str.contains(diverse) | df['description'].str.contains(diverse) | df['description2'].str.contains(diverse) | df['title'].str.contains(diverse)]
diversity_library_shows= df_2[df_2['tags2'].str.contains(diverse) | df_2['description'].str.contains(diverse) | df_2['description2'].str.contains(diverse) | df_2['title'].str.contains(diverse)
]

In [18]:
#reset index for df_2
df_2.reset_index(inplace=True)
#drop the column 'index'
df_2.drop(columns=['index'], inplace=True)
df_2.head()

,level_0,id,title,category,description,description2,directors_actors,publication_date,rating,duration_sec,tags,tags2,image,features,cluster
0,0,10000,199 Little Heroes,Education,The journey to school is a very special type o...,"[Enjo lives in Quinten, a forest glade in the ...",[[]],2021-06-01 06:20:00,G,"[311.0, 311.0, 312.0, 311.0, 311.0, 320.0, 311...","[ABC ME, EDUCATION]","[primary-humanities, education, abc3, refugee-...",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW073...,journey school special type life experience sh...,7
1,1,10001,7.30,News,Leigh Sales presents Australia's leading night...,[The devastation caused by the recent floods i...,[[]],2022-03-14 20:00:00,None,"[1906.0, 1894.0, 1879.0, 1935.0, 1891.0, 1822....","[ABC TV, ABC NEWS]","[lismore, news, childcare, flooding, abc-faves...",https://cdn.iview.abc.net.au/thumbs/i/nc/NC220...,leigh sale present australia leading nightly p...,1
2,2,10002,7.30 Mark Humphries Satire,News,Satirist Mark Humphries brings his unique pers...,[Satirist Mark Humphries goes inside the Healt...,[[]],2021-05-11 14:00:00,None,"[145.0, 110.0, 118.0, 136.0, 202.0, 115.0, 241...",[ABC NEWS],"[mark, parachuting, humphries, candidates, sus...",https://cdn.iview.abc.net.au/thumbs/i/nn/NN211...,satirist mark humphries brings unique perspect...,2
3,3,10003,7.30: The Interviews,News,Revisit some of 7.30's most compelling convers...,[Musician Dave Grohl sits down with 7.30's Lei...,[[]],2022-03-08 20:00:00,None,[1030.0],[ABC NEWS],"[news24, news]",https://cdn.iview.abc.net.au/thumbs/i/ns/NS224...,revisit 730s compelling conversation australia...,5
4,4,10004,72 Cutest Animals,Family,The animal kingdom continually astounds us wit...,[Cute is a notion that can mean many things to...,[[]],2022-03-05 15:03:06,G,"[1616.0, 1616.0, 1620.0, 1620.0, 1614.0, 1620....","[ABC ME, FAMILY]","[family-viewing, family-viewing-abcme, abc3]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW243...,animal kingdom continually astounds u showcase...,0


In [19]:
# Print the top 10 terms for each cluster
order_centroids = km.cluster_centers_.argsort()[:, ::-1]
terms = tfidf_vectorizer.get_feature_names()
for i in range(num_clusters):
    print("Cluster %d:" % i, end='')
    for ind in order_centroids[i, :13]:
        print(' %s' % terms[ind], end='')
    print()



Cluster 0: people disability christmas beach country story chinese like day new live lost parent
Cluster 1: world come journey ultimate epic magical greatest earth life new deep book great
Cluster 2: artist political art visual corrupt personal universe australia history whitewashing exile regime abuse
Cluster 3: love couple family life 2010 follow stick story begin friend affair new tom
Cluster 4: human attenborough year david sir truth experiment home set new family really animal
Cluster 5: performed ballet australia opera orchestra australian sydney music love present victoria woman tale
Cluster 6: story tell life stage english australia left help set film extraordinary man remarkable
Cluster 7: win passion 2016 raising state united competition best encounter chance hidden biker young
Cluster 8: school make play hand safe family new special wash child episode stay explain
Cluster 9: young try power 2013 psychic starring action international life 2011 physician protect katrina
Cluste

/Users/ekaterinamazur/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [20]:
# Print the top 10 terms for each cluster
order_centroids_shows = km_shows.cluster_centers_.argsort()[:, ::-1]
terms_shows = tfidf_vectorizer_shows.get_feature_names()
for i in range(num_clusters_shows):
    print("Cluster %d:" % i, end='')
    for ind in order_centroids_shows[i, :13]:
        print(' %s' % terms_shows[ind], end='')
    print()

Cluster 0: life series history story new world people australia art child tell join friend
Cluster 1: news abc issue story reporting brings today original exclusive presented investigation matter live
Cluster 2: team question awardwinning game robot story australian current new abc crime series war
Cluster 3: adventure set friend town series feature action inspired character life live explore comedy
Cluster 4: science australian teacher skill australia teach lesson fundamental tenminute journey family series weather
Cluster 5: big world friend animal sporting best old sport adventure issue business way school
Cluster 6: book love learn share bear adventure language friend hungry australia new family world
Cluster 7: stage english math early life world series geography 45 science history word pdhpe


/Users/ekaterinamazur/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [21]:
joined_activity = pd.merge(df, activities, left_on='id', right_on='content_id', how='right')
#make from list in tag2 column to string and delete [] in the string
joined_activity['tags2'] = joined_activity['tags2'].apply(lambda x: str(x).replace('[', '').replace(']', ''))
#drop NaN values in id column
joined_activity.dropna(subset=['id'], inplace=True)
#drop columns that are not needed
joined_activity.drop(['series', 'episode_name', 'description2', 'publication_date', 'rating', 'season', 'episode', 'episode_title', 'datetime', 'duration_sec', 'tags'], axis=1, inplace=True)
joined_activity.head()

,id,category,title,description,image,directors_actors,tags2,features,cluster,content_id,activity,user_id
3,144.0,Family,Zog,"Based on the picture book by Julia Donaldson, ...",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW124...,"[['Director Max Lang', 'Daniel Snaddon']]","'abc4kids', '4feature', '4featured', 'abcme-sh...",based picture book julia donaldson animated co...,9.0,144,Dislike,14
12,246.0,Documentary,Tea with the Dames,"Screen icons Eileen Atkins, Judi Dench, Joan P...",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW188...,[['Director Roger Michell']],"'abc1', 'uk', 'docs', 'factual', 'biography', ...",screen icon eileen atkins judi dench joan plow...,12.0,246,Play,10
21,34.0,Movies,Like Minds,"When a schoolboy is suspected of murder, a for...",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW290...,[['Director Gregory J. Read']],"'abc1', 'abc2', 'thriller', 'drama', 'crime', ...",schoolboy suspected murder forensic psychologi...,11.0,34,Dislike,15
24,219.0,Documentary,I'm Wanita,"The story of Australia's Queen of Honky Tonk, ...",https://cdn.iview.abc.net.au/thumbs/i/ac/AC182...,[[]],"'abc2', 'abc1', 'docs', 'factual', 'abcarts', ...",story australia queen honky tonk wanita bahtiy...,6.0,219,Like,11
26,328.0,Comedy,The Pirates Of Penzance (1994),This spectacular production of the popular Gil...,https://cdn.iview.abc.net.au/thumbs/i/ae/AE941...,[[]],"'abc2', 'abcarts', 'australia', 'arts', 'class...",spectacular production popular gilbert sulliva...,6.0,328,Dislike,16


In [22]:
joined_activity_shows = pd.merge(df_2, activities, left_on='id', right_on='content_id', how='right')
#make from list in tag2 column to string and delete [] in the string
joined_activity_shows['tags2'] = joined_activity_shows['tags2'].apply(lambda x: str(x).replace('[', '').replace(']', ''))
#drop NaN values in id column
joined_activity_shows.dropna(subset=['id'], inplace=True)
#drop columns that are not needed
joined_activity_shows.drop([ 'description2', 'publication_date', 'rating', 'datetime', 'duration_sec', 'tags'], axis=1, inplace=True)
joined_activity_shows.head()

,level_0,id,title,category,description,directors_actors,tags2,image,features,cluster,content_id,activity,user_id
0,143.0,10143.0,Decoding Danger,Documentary,"Apocalyptic fire storms, raging floods and the...",[[]],"'science', 'informative', 'animals', 'climate'...",https://cdn.iview.abc.net.au/thumbs/i/do/DO195...,apocalyptic fire storm raging flood venomous c...,4.0,10143,Like,14
1,78.0,10078.0,Bed Of Roses,Drama,An Australian comedy drama which centres on Lo...,"['Julia Blake', 'Hanna Mangan Lawrence', ['Cas...","'a:older-sink-dink', 'love', 'comedy', 'abc1',...",https://cdn.iview.abc.net.au/thumbs/i/dr/publi...,australian comedy drama centre louisa atherton...,3.0,10078,Like,11
2,227.0,10227.0,Hitting Home With Sarah Ferguson,Documentary,Award-winning journalist Sarah Ferguson spends...,[['Host Sarah Ferguson']],"'ns:be-informed', 'conversation-starters', 'a:...",https://cdn.iview.abc.net.au/thumbs/i/do/DO140...,awardwinning journalist sarah ferguson spends ...,2.0,10227,Play,14
4,553.0,10553.0,The Wonder Of Marsupials,Family,From Australia's scorching desert heart to pri...,"['Edward Saltau'], ['Director Daniel Hunter']","'ns:be-informed', 'natural-world', 'compelling...",https://cdn.iview.abc.net.au/thumbs/i/ai/AI185...,australia scorching desert heart pristine rain...,3.0,10553,Like,20
5,343.0,10343.0,Muster Dogs,Family,Five graziers from across Australia are given ...,[[]],"'endearing', 'real-life', 'diversity', 'region...",https://cdn.iview.abc.net.au/thumbs/i/rf/RF200...,five grazier across australia given five kelpi...,3.0,10343,Play,16


In [23]:
#select rows where activity is 'Like'
expat_data_likes = joined_activity[joined_activity['activity'] == 'Like']
expat_data_likes.sort_values(by=['user_id'])

,id,category,title,description,image,directors_actors,tags2,features,cluster,content_id,activity,user_id
5835,58.0,Movies,Angel,"Adapted from Elizabeth Taylor's novel, a passi...",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW287...,[['Director Francois Ozon']],"'abc1', 'abc2', 'uk', 'europe', 'drama', 'roma...",adapted elizabeth taylor novel passionate youn...,11.0,58,Like,0
4135,30.0,Movies,The Well,Best friends raised on an isolated farm are lo...,https://cdn.iview.abc.net.au/thumbs/i/zy/ZY946...,[['Director Samantha Lang']],"'abc1', 'drama', 'film', '1990s', 'countryside...",best friend raised isolated farm looking futur...,7.0,30,Like,0
5405,20.0,Movies,Five Easy Pieces,After rejecting his wealthy and cultured upbri...,https://cdn.iview.abc.net.au/thumbs/i/zy/ZY928...,[['Director Bob Rafelson']],"'abc1', 'abc2', 'usa', 'americana', '1970s', '...",rejecting wealthy cultured upbringing young ma...,11.0,20,Like,0
5153,337.0,Education,Carbon Cycles And Climate Change In The Tundra,"Filmed in the Canadian Arctic, this program id...",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW193...,[[]],"'abc3', 'education', 'secondary-geography'",filmed canadian arctic program identifies expl...,10.0,337,Like,0
6347,319.0,Comedy,Opera On Sydney Harbour: La Boheme,Experience the romance of the original bohemia...,https://cdn.iview.abc.net.au/thumbs/i/zw/ZW230...,[['Director Andy Morton']],"'abc2', 'abcarts', 'arts', 'opera', 'performan...",experience romance original bohemian love stor...,5.0,319,Like,0
...,...,...,...,...,...,...,...,...,...,...,...,...
6429,163.0,Family,Legends Of Oz: Dorothy's Return,An animated musical based on the adventure boo...,https://cdn.iview.abc.net.au/thumbs/i/zw/ZW041...,"[['Director Daniel St. Pierre', 'Will Finn']]","'abc3', 'childrens', '3featured', 'abc3-faves'...",animated musical based adventure book roger st...,11.0,163,Like,32
6023,155.0,Family,Australia: The Wild Top End,Come with us on an epic journey through some o...,https://cdn.iview.abc.net.au/thumbs/i/zw/ZW284...,[[]],"'abc3', '3featured', 'family-viewing-abcme', '...",come u epic journey wildest landscape earth ta...,1.0,155,Like,32
6271,145.0,Family,Armadillo: Narrated By David Attenborough,In the heart of Brazil lives an animal so elus...,https://cdn.iview.abc.net.au/thumbs/i/zw/ZW311...,[['Host Sir David Attenborough']],"'abc1', 'uk', 'docs', 'factual', 'nature', 'wi...",heart brazil life animal elusive ever seen wil...,4.0,145,Like,32
6136,34.0,Movies,Like Minds,"When a schoolboy is suspected of murder, a for...",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW290...,[['Director Gregory J. Read']],"'abc1', 'abc2', 'thriller', 'drama', 'crime', ...",schoolboy suspected murder forensic psychologi...,11.0,34,Like,32


In [24]:
#select rows where activity is 'Like'
expat_data_shows_likes = joined_activity_shows[joined_activity_shows['activity'] == 'Like']
expat_data_shows_likes.sort_values(by=['user_id'])

,level_0,id,title,category,description,directors_actors,tags2,image,features,cluster,content_id,activity,user_id
7055,103.0,10103.0,Buddi,Kids,Buddi welcomes you into a magical world filled...,[[]],"'abc4kids', '4puggle'",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW240...,buddi welcome magical world filled natural won...,0.0,10103,Like,0
7137,467.0,10467.0,Spicks And Specks,Panel Discussion,"Join Adam Hills, Myf Warhurst and Alan Brough ...","['Myf Warhurst', 'Alan Brough'], ['Hosts Adam ...","'ns:be-entertained', 'a:older-sink-dink', 'wit...",https://cdn.iview.abc.net.au/thumbs/i/le/LE200...,join adam hill myf warhurst alan brough brand ...,2.0,10467,Like,0
4111,512.0,10512.0,The Inbestigators,Family,"When ten-year-old Maudie, a freakishly gifted ...","[['Cast Anna Cooke', 'Jamil Smyth-Secka', 'Ast...","'family-viewing-abcme', 'explore', 'abc3-faves...",https://cdn.iview.abc.net.au/thumbs/i/ck/CK190...,tenyearold maudie freakishly gifted observer h...,0.0,10512,Like,0
6779,580.0,10580.0,Vera,Drama,"DCI Vera Stanhope returns, leading her diligen...","['Kenny Doughty', 'Jon Morrison', 'Ibinabo Jac...","'mystery', 'compelling', 'thriller', 'strong-f...",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW314...,dci vera stanhope return leading diligent team...,2.0,10580,Like,0
6985,451.0,10451.0,Sesame Street,Kids,"Join Big Bird, Oscar, Elmo, Bert, Cookie Monst...",[[]],"'happy', '4celebrity', '4best', 'childrens', '...",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW162...,join big bird oscar elmo bert cookie monster c...,3.0,10451,Like,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3107,270.0,10270.0,Kids Listen: Story Salad,Kids,Join Matt from Play School and one of his stor...,[[]],'abc4kids',https://cdn.iview.abc.net.au/thumbs/i/ch/CH204...,join matt play school one storyteller friend g...,0.0,10270,Like,30
3217,564.0,10564.0,Tish Tash,Kids,Follow the adventures of a young bear called T...,[[]],"'4girls', 'abc4kids'",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW276...,follow adventure young bear called tish bear f...,6.0,10564,Like,30
3256,507.0,10507.0,The Heights,Drama,Welcome to Arcadia Heights. Whether it's the h...,"['Koa Nuen', ['Cast Marcus Graham', 'Shari Seb...","'relationship-drama', 'love', 'coming-of-age',...",https://cdn.iview.abc.net.au/thumbs/i/dr/DR170...,welcome arcadia height whether highrise high l...,7.0,10507,Like,30
2670,404.0,10404.0,Play School: Through The Windows,Kids,"Through the eyes of a child, explore the world...",[[]],"'christmas', '4pslets-explore', '4aussie', '4u...",https://cdn.iview.abc.net.au/thumbs/i/ck/CK180...,eye child explore world around u full humour w...,0.0,10404,Like,30


In [25]:
#top-3 clusters for each user in expat_data_likes
top3 = expat_data_likes.groupby('user_id')['cluster'].value_counts().groupby(level=0).nlargest(3).reset_index(level=1, drop=True)
#make from series to dataframe
top3 = top3.to_frame()
#drop the 3rd column
top3 = top3.drop(columns=['cluster'])
#make user_id column as index
top3 = top3.reset_index()
top3

,user_id,cluster
0,0,1.0
1,0,5.0
2,0,7.0
3,1,6.0
4,1,5.0
...,...,...
93,31,6.0
94,31,5.0
95,32,11.0
96,32,1.0


In [26]:
expat_data_shows_likes.sort_values(by=['user_id'])
#top-3 clusters for each user in expat_data_likes
top3_show = expat_data_shows_likes.groupby('user_id')['cluster'].value_counts().groupby(level=0).nlargest(3).reset_index(level=1,
                                                                                                              drop=True)
#make from series to dataframe
top3_show = top3_show.to_frame()
#drop the 3rd column
top3_show = top3_show.drop(columns=['cluster'])
#make user_id column as index
top3_show = top3_show.reset_index()
top3_show

,user_id,cluster
0,0,0.0
1,0,3.0
2,0,5.0
3,1,0.0
4,1,7.0
...,...,...
88,29,3.0
89,29,4.0
90,30,0.0
91,30,3.0


In [27]:
#join top 3 clusters for each user with expat_library
recommended= pd.merge(top3, expat_library, on=['cluster'], how='left')
recommended

,user_id,cluster,episode_title,id,category,title,series,episode_name,description,description2,tags,image,directors_actors,tags2,publication_date,rating,duration_sec,season,episode,features
0,0,1.0,Can We Save The Reef?,152,Family,Can We Save The Reef?,NaN,Can We Save The Reef?,An epic story of Australian and international ...,An epic story of Australian and international ...,"[ABC TV, DOCUMENTARY, SCIENCE, FAMILY]",https://cdn.iview.abc.net.au/thumbs/i/sc/SC160...,[[]],"[abc1, docs, nature, science, science-week, cl...",2018-05-12 14:58:55,G,3428.0,NaN,,epic story australian international scientist ...
1,0,1.0,Australia: The Wild Top End,155,Family,Australia: The Wild Top End,NaN,Australia: The Wild Top End,Come with us on an epic journey through some o...,Come with us on an epic journey through some o...,"[ABC ME, FAMILY]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW284...,[[]],"[abc3, 3featured, family-viewing-abcme, family...",2022-03-16 09:45:00,PG,2602.0,NaN,,come u epic journey wildest landscape earth ta...
2,0,1.0,Sea Lions: Life By A Whisker,187,Family,Sea Lions: Life By A Whisker,NaN,Sea Lions: Life By A Whisker,Take an intimate journey inside the colony of ...,Take an intimate journey inside the colony of ...,"[ABC ME, FAMILY]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW284...,[['Narrator Sam Neill']],"[abc3, family-viewing-abcme, 3featured, family...",2022-03-16 09:45:00,G,2439.0,NaN,,take intimate journey inside colony australian...
3,0,1.0,Bronwyn Oliver: The Shadows Within,243,Documentary,Bronwyn Oliver: The Shadows Within,NaN,Bronwyn Oliver: The Shadows Within,This intimate documentary follows the journey ...,This intimate documentary follows the journey ...,"[ABC TV Plus, ABC ARTS, ARTS & CULTURE, DOCUME...",https://cdn.iview.abc.net.au/thumbs/i/ac/AC201...,[[]],"[abc2, arts, abcarts, docs, factual, new-south...",2021-10-13 21:03:54,PG,3567.0,NaN,,intimate documentary follows journey bronwyn o...
4,0,5.0,Children Of The Revolution,112,Movies,Children Of The Revolution,NaN,Children Of The Revolution,Few knew that Stalin spent his last night in t...,Few knew that Stalin spent his last night in t...,"[ABC TV, DRAMA, MOVIES, COMEDY]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW303...,"[['Cast Judy Davis', 'Richard Roxburgh', 'Sam ...","[abc1, drama, film, comedy, australia, politic...",2021-10-01 07:00:00,M,6028.0,NaN,,knew stalin spent last night arm young austral...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
677,32,1.0,Can We Save The Reef?,152,Family,Can We Save The Reef?,NaN,Can We Save The Reef?,An epic story of Australian and international ...,An epic story of Australian and international ...,"[ABC TV, DOCUMENTARY, SCIENCE, FAMILY]",https://cdn.iview.abc.net.au/thumbs/i/sc/SC160...,[[]],"[abc1, docs, nature, science, science-week, cl...",2018-05-12 14:58:55,G,3428.0,NaN,,epic story australian international scientist ...
678,32,1.0,Australia: The Wild Top End,155,Family,Australia: The Wild Top End,NaN,Australia: The Wild Top End,Come with us on an epic journey through some o...,Come with us on an epic journey through some o...,"[ABC ME, FAMILY]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW284...,[[]],"[abc3, 3featured, family-viewing-abcme, family...",2022-03-16 09:45:00,PG,2602.0,NaN,,come u epic journey wildest landscape earth ta...
679,32,1.0,Sea Lions: Life By A Whisker,187,Family,Sea Lions: Life By A Whisker,NaN,Sea Lions: Life By A Whisker,Take an intimate journey inside the colony of ...,Take an intimate journey inside the colony of ...,"[ABC ME, FAMILY]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW284...,[['Narrator Sam Neill']],"[abc3, family-viewing-abcme, 3featured, family...",2022-03-16 09:45:00,G,2439.0,NaN,,take intimate journey inside colony australian...
680,32,1.0,Bronwyn Oliver: The Shadows Within,243,Documentary,Bronwyn Oliver: The Shadows Within,NaN,Bronwyn Oliver: The Shadows Within,This intimate documentary follows the journey ...,This intimate documen

In [28]:
#shows
recommended_shows= pd.merge(top3_show, expat_library_shows, on=['cluster'], how='left')
recommended_shows

,user_id,cluster,index,id,title,category,description,description2,directors_actors,publication_date,rating,duration_sec,tags,tags2,image,features
0,0,0.0,52.0,10052.0,Australia Remastered: Australia's Oceans,Family,Explore the incredible diversity of ocean wild...,[The Southern Ocean is the only ocean that str...,[['Host Aaron Pedersen']],2020-10-25 19:03:00,G,"[3230.0, 3195.0, 3185.0]","[ABC TV, ABC ME, DOCUMENTARY, SCIENCE, FAMILY]","[ns:be-informed, natural-world, wildlife, a:ol...",https://cdn.iview.abc.net.au/thumbs/i/do/DO184...,explore incredible diversity ocean wildlife en...
1,0,0.0,55.0,10055.0,Australia Remastered: Wild Australians,Family,Explore some of Australia's most iconic and fa...,"[Echidnas and platypuses are unique, the only ...",[['Host Aaron Pedersen']],2020-08-30 18:57:00,G,"[3310.0, 3330.0, 3285.0, 3185.0, 3230.0, 3294.0]","[ABC TV, ABC ME, DOCUMENTARY, FAMILY]","[ns:be-informed, natural-world, wildlife, a:ol...",https://cdn.iview.abc.net.au/thumbs/i/do/DO184...,explore australia iconic fascinating animal my...
2,0,0.0,56.0,10056.0,Australia Remastered: Wild Treasures,Family,Australia's diverse and unique landscapes and ...,[The Daintree Rainforest has one of the highes...,[['Host Aaron Pedersen']],2021-01-11 21:22:47,G,"[3205.0, 3315.0, 3260.0, 3254.0, 3257.0, 3220.0]","[ABC TV, ABC ME, DOCUMENTARY, FAMILY]","[natural-world, wildlife, family-viewing-abcme...",https://cdn.iview.abc.net.au/thumbs/i/do/DO184...,australia diverse unique landscape ecosystem u...
3,0,0.0,57.0,10057.0,Australia Remastered: Wild Wars,Family,Life in Australia is a battle. Shaped by the c...,[While forests only cover 16% of Australia's l...,[['Host Aaron Pedersen']],2021-03-02 22:33:27,G,"[3235.0, 3265.0, 3280.0, 3240.0]","[ABC TV, DOCUMENTARY, FAMILY]","[ns:be-informed, natural-world, wildlife, a:ol...",https://cdn.iview.abc.net.au/thumbs/i/do/DO184...,life australia battle shaped challenge epic en...
4,0,0.0,144.0,10144.0,Deep Dive Into Australia's Ocean Odyssey,Education,"Focussing on marine life, interconnectedness o...",[Emily Jateff from Australian National Maritim...,[[]],2020-08-17 11:56:21,PG,"[1206.0, 1090.0, 1271.0]","[ABC ME, EDUCATION]","[secondary-maths, education, abc3]",https://cdn.iview.abc.net.au/thumbs/i/ck/CK203...,focussing marine life interconnectedness land ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
339,30,0.0,57.0,10057.0,Australia Remastered: Wild Wars,Family,Life in Australia is a battle. Shaped by the c...,[While forests only cover 16% of Australia's l...,[['Host Aaron Pedersen']],2021-03-02 22:33:27,G,"[3235.0, 3265.0, 3280.0, 3240.0]","[ABC TV, DOCUMENTARY, FAMILY]","[ns:be-informed, natural-world, wildlife, a:ol...",https://cdn.iview.abc.net.au/thumbs/i/do/DO184...,life australia battle shaped challenge epic en...
340,30,0.0,144.0,10144.0,Deep Dive Into Australia's Ocean Odyssey,Education,"Focussing on marine life, interconnectedness o...",[Emily Jateff from Australian National Maritim...,[[]],2020-08-17 11:56:21,PG,"[1206.0, 1090.0, 1271.0]","[ABC ME, EDUCATION]","[secondary-maths, education, abc3]",https://cdn.iview.abc.net.au/thumbs/i/ck/CK203...,focussing marine life interconnectedness land ...
341,30,0.0,485.0,10485.0,The Art Of Australia,Documentary,The Art of Australia explores how art and arti...,[The story of how art helped European settlers...,[['Host Edmund Capon']],2021-06-18 07:00:00,G,"[3390.0, 3423.0, 3402.0]","[ABC TV, ABC ARTS, ARTS & CULTURE, DOCUMENTARY]","[informative, a:older-sink-dink, ns:be-inspire...",https://cdn.iview.abc.net.au/thumbs/i/ac/AC123...,art australia explores art artist helped shape...
342,30,3.0,210.0,10210.0,Griff's Great Australian Rail Trip,Documentary,British comedian and travel enthusiast Griff R...,[Journeying from Brisbane to Cairns aboard the...,[['Host Griff Rhys Jones']],2021-06-30 07:00:00,G,"[2829.0, 2785.0, 2831.0, 2768.0, 2821.0, 2832.0]","[ABC TV, LIFESTYLE, DOCUMENTARY]","[factual, travel, lifestyle, abc1, uk, austra

In [29]:
recommended_div= pd.merge(top3, diversity_library, on=['cluster'], how='left')
recommended_shows_div= pd.merge(top3_show, diversity_library_shows, on=['cluster'], how='left')
recommended_div

,user_id,cluster,episode_title,id,category,title,series,episode_name,description,description2,tags,image,directors_actors,tags2,publication_date,rating,duration_sec,season,episode,features
0,0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,5.0,Australia Day Live,313.0,Comedy,Australia Day Live 2022,NaN,Australia Day Live 2022,A concert and fireworks spectacular from Sydne...,A concert and fireworks spectacular from Sydne...,"[ABC TV, ABC ARTS, ARTS & CULTURE]",https://cdn.iview.abc.net.au/thumbs/i/rv/RV210...,"[['Host Jeremy Fernandez', 'Casey Donovan', 'J...","[abc1, aussie, australia, event, concert, musi...",2022-01-26 21:31:00,PG,7871.0,NaN,,concert firework spectacular sydney featuring ...
2,0,5.0,Ochres,326.0,Comedy,Ochres,NaN,Ochres,"Told in five parts, Ochres was inspired by the...","Told in five parts, Ochres was inspired by the...","[ABC ARTS, ARTS & CULTURE, INDIGENOUS]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW229...,[[]],"[abcarts, arts, dance, bangarra, first-nations...",2021-06-18 07:00:00,G,3417.0,NaN,,told five part ochre inspired special role och...
3,0,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182,31,5.0,Australia Day Live,313.0,Comedy,Australia Day Live 2022,NaN,Australia Day Live 2022,A concert and fireworks spectacular from Sydne...,A concert and fireworks spectacular from Sydne...,"[ABC TV, ABC ARTS, ARTS & CULTURE]",https://cdn.iview.abc.net.au/thumbs/i/rv/RV210...,"[['Host Jeremy Fernandez', 'Casey Donovan', 'J...","[abc1, aussie, australia, event, concert, musi...",2022-01-26 21:31:00,PG,7871.0,NaN,,concert firework spectacular sydney featuring ...
183,31,5.0,Ochres,326.0,Comedy,Ochres,NaN,Ochres,"Told in five parts, Ochres was inspired by the...","Told in five parts, Ochres was inspired by the...","[ABC ARTS, ARTS & CULTURE, INDIGENOUS]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW229...,[[]],"[abcarts, arts, dance, bangarra, first-nations...",2021-06-18 07:00:00,G,3417.0,NaN,,told five part ochre inspired special role och...
184,32,11.0,The Broken Shore,92.0,Movies,The Broken Shore,NaN,The Broken Shore,An evocative crime thriller set against a back...,An evocative crime thriller set against a back...,"[ABC TV, ABC TV Plus, DRAMA, MOVIES, MOVIES]",https://cdn.iview.abc.net.au/thumbs/i/dr/DR121...,[['Director Rowan Woods']],"[abc1, abc2, drama, crime, film, murder, aussi...",2021-06-11 07:05:00,M,6212.0,NaN,,evocative crime thriller set background police...
185,32,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
#drop intersections of recommended and activity by user_id and content_id
recommended = recommended[~recommended[['user_id', 'id']].apply(frozenset, axis=1).isin(activities[['user_id', 'content_id']].apply(frozenset, axis=1))]
recommended

,user_id,cluster,episode_title,id,category,title,series,episode_name,description,description2,tags,image,directors_actors,tags2,publication_date,rating,duration_sec,season,episode,features
0,0,1.0,Can We Save The Reef?,152,Family,Can We Save The Reef?,NaN,Can We Save The Reef?,An epic story of Australian and international ...,An epic story of Australian and international ...,"[ABC TV, DOCUMENTARY, SCIENCE, FAMILY]",https://cdn.iview.abc.net.au/thumbs/i/sc/SC160...,[[]],"[abc1, docs, nature, science, science-week, cl...",2018-05-12 14:58:55,G,3428.0,NaN,,epic story australian international scientist ...
2,0,1.0,Sea Lions: Life By A Whisker,187,Family,Sea Lions: Life By A Whisker,NaN,Sea Lions: Life By A Whisker,Take an intimate journey inside the colony of ...,Take an intimate journey inside the colony of ...,"[ABC ME, FAMILY]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW284...,[['Narrator Sam Neill']],"[abc3, family-viewing-abcme, 3featured, family...",2022-03-16 09:45:00,G,2439.0,NaN,,take intimate journey inside colony australian...
3,0,1.0,Bronwyn Oliver: The Shadows Within,243,Documentary,Bronwyn Oliver: The Shadows Within,NaN,Bronwyn Oliver: The Shadows Within,This intimate documentary follows the journey ...,This intimate documentary follows the journey ...,"[ABC TV Plus, ABC ARTS, ARTS & CULTURE, DOCUME...",https://cdn.iview.abc.net.au/thumbs/i/ac/AC201...,[[]],"[abc2, arts, abcarts, docs, factual, new-south...",2021-10-13 21:03:54,PG,3567.0,NaN,,intimate documentary follows journey bronwyn o...
4,0,5.0,Children Of The Revolution,112,Movies,Children Of The Revolution,NaN,Children Of The Revolution,Few knew that Stalin spent his last night in t...,Few knew that Stalin spent his last night in t...,"[ABC TV, DRAMA, MOVIES, COMEDY]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW303...,"[['Cast Judy Davis', 'Richard Roxburgh', 'Sam ...","[abc1, drama, film, comedy, australia, politic...",2021-10-01 07:00:00,M,6028.0,NaN,,knew stalin spent last night arm young austral...
5,0,5.0,Romeo & Juliet,148,Family,Romeo & Juliet,NaN,Romeo & Juliet,"Graeme Murphy's Romeo and Juliet, performed by...","Graeme Murphy's Romeo and Juliet, performed by...","[ABC TV Plus, ABC ARTS, ARTS & CULTURE, FAMILY]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW230...,[[]],"[abc2, arts, abcarts, ballet, entertainment, c...",2021-06-18 07:00:00,PG,7168.0,NaN,,graeme murphy romeo juliet performed australia...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
676,32,11.0,Mark Seymour And The Undertow: Slow Dawn Live,315,Comedy,Mark Seymour And The Undertow: Slow Dawn Live,NaN,Mark Seymour And The Undertow: Slow Dawn Live,Hunters & Collectors' frontman Mark Seymour pr...,Hunters & Collectors' frontman Mark Seymour pr...,"[ABC TV Plus, ARTS & CULTURE]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW312...,[[]],"[abc2, arts, australia, music, performance, po...",2021-11-05 20:31:33,PG,2960.0,NaN,,hunter collector frontman mark seymour present...
677,32,1.0,Can We Save The Reef?,152,Family,Can We Save The Reef?,NaN,Can We Save The Reef?,An epic story of Australian and international ...,An epic story of Australian and international ...,"[ABC TV, DOCUMENTARY, SCIENCE, FAMILY]",https://cdn.iview.abc.net.au/thumbs/i/sc/SC160...,[[]],"[abc1, docs, nature, science, science-week, cl...",2018-05-12 14:58:55,G,3428.0,NaN,,epic story australian international scientist ...
679,32,1.0,Sea Lions: Life By A Whisker,187,Family,Sea Lions: Life By A Whisker,NaN,Sea Lions: Life By A Whisker,Take an intimate journey inside the colony of ...,Take an intimate journey inside the colony of ...,"[ABC ME, FAMILY]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW284...,[['Narrator Sam Neill']],"[abc3, family-viewing-abcme, 3featured, family...",2022-03-16 09:45:00,G,2439.0,NaN,,take intimate journey inside colony australian...
680,32,1.0,Bronwyn Oliver: The Shadows Within,243,Documentary,Bronwyn Oliver: The Shadows Within,NaN,Bronwyn Oliver: The Shadows Within,This in

In [31]:
recommended_shows = recommended_shows[~recommended_shows[['user_id', 'id']].apply(frozenset, axis=1).isin(activities[['user_id', 'content_id']].apply(frozenset, axis=1))]
recommended_shows

,user_id,cluster,index,id,title,category,description,description2,directors_actors,publication_date,rating,duration_sec,tags,tags2,image,features
0,0,0.0,52.0,10052.0,Australia Remastered: Australia's Oceans,Family,Explore the incredible diversity of ocean wild...,[The Southern Ocean is the only ocean that str...,[['Host Aaron Pedersen']],2020-10-25 19:03:00,G,"[3230.0, 3195.0, 3185.0]","[ABC TV, ABC ME, DOCUMENTARY, SCIENCE, FAMILY]","[ns:be-informed, natural-world, wildlife, a:ol...",https://cdn.iview.abc.net.au/thumbs/i/do/DO184...,explore incredible diversity ocean wildlife en...
2,0,0.0,56.0,10056.0,Australia Remastered: Wild Treasures,Family,Australia's diverse and unique landscapes and ...,[The Daintree Rainforest has one of the highes...,[['Host Aaron Pedersen']],2021-01-11 21:22:47,G,"[3205.0, 3315.0, 3260.0, 3254.0, 3257.0, 3220.0]","[ABC TV, ABC ME, DOCUMENTARY, FAMILY]","[natural-world, wildlife, family-viewing-abcme...",https://cdn.iview.abc.net.au/thumbs/i/do/DO184...,australia diverse unique landscape ecosystem u...
3,0,0.0,57.0,10057.0,Australia Remastered: Wild Wars,Family,Life in Australia is a battle. Shaped by the c...,[While forests only cover 16% of Australia's l...,[['Host Aaron Pedersen']],2021-03-02 22:33:27,G,"[3235.0, 3265.0, 3280.0, 3240.0]","[ABC TV, DOCUMENTARY, FAMILY]","[ns:be-informed, natural-world, wildlife, a:ol...",https://cdn.iview.abc.net.au/thumbs/i/do/DO184...,life australia battle shaped challenge epic en...
4,0,0.0,144.0,10144.0,Deep Dive Into Australia's Ocean Odyssey,Education,"Focussing on marine life, interconnectedness o...",[Emily Jateff from Australian National Maritim...,[[]],2020-08-17 11:56:21,PG,"[1206.0, 1090.0, 1271.0]","[ABC ME, EDUCATION]","[secondary-maths, education, abc3]",https://cdn.iview.abc.net.au/thumbs/i/ck/CK203...,focussing marine life interconnectedness land ...
7,0,5.0,290.0,10290.0,Little J And Big Cuz Indigenous Languages,Kids,Little J and Big Cuz are Indigenous Australian...,[Nanna is unwell so Little J and Big Cuz eager...,[[]],2021-07-31 06:00:00,G,"[760.0, 778.0, 706.0, 704.0, 720.0, 760.0, 727...","[ABC Kids, EDUCATION]","[littlejin, abc4kids, education]",https://cdn.iview.abc.net.au/thumbs/i/ck/CK194...,little j big cuz indigenous australian kid liv...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
338,30,0.0,56.0,10056.0,Australia Remastered: Wild Treasures,Family,Australia's diverse and unique landscapes and ...,[The Daintree Rainforest has one of the highes...,[['Host Aaron Pedersen']],2021-01-11 21:22:47,G,"[3205.0, 3315.0, 3260.0, 3254.0, 3257.0, 3220.0]","[ABC TV, ABC ME, DOCUMENTARY, FAMILY]","[natural-world, wildlife, family-viewing-abcme...",https://cdn.iview.abc.net.au/thumbs/i/do/DO184...,australia diverse unique landscape ecosystem u...
339,30,0.0,57.0,10057.0,Australia Remastered: Wild Wars,Family,Life in Australia is a battle. Shaped by the c...,[While forests only cover 16% of Australia's l...,[['Host Aaron Pedersen']],2021-03-02 22:33:27,G,"[3235.0, 3265.0, 3280.0, 3240.0]","[ABC TV, DOCUMENTARY, FAMILY]","[ns:be-informed, natural-world, wildlife, a:ol...",https://cdn.iview.abc.net.au/thumbs/i/do/DO184...,life australia battle shaped challenge epic en...
341,30,0.0,485.0,10485.0,The Art Of Australia,Documentary,The Art of Australia explores how art and arti...,[The story of how art helped European settlers...,[['Host Edmund Capon']],2021-06-18 07:00:00,G,"[3390.0, 3423.0, 3402.0]","[ABC TV, ABC ARTS, ARTS & CULTURE, DOCUMENTARY]","[informative, a:older-sink-dink, ns:be-inspire...",https://cdn.iview.abc.net.au/thumbs/i/ac/AC123...,art australia explores art artist helped shape...
342,30,3.0,210.0,10210.0,Griff's Great Australian Rail Trip,Documentary,British comedian and travel enthusiast Griff R...,[Journeying from Brisbane to Cairns aboard the...,[['Host Griff Rhys Jones']],2021-06-30 07:00:00,G,"[2829.0, 2785.0, 2831.0, 2768.0, 2821.0, 2832.0]","[ABC TV, LIFESTYLE, DOCUMENTARY]","[factual, travel, lifestyle, ab

In [32]:
recommended_div = recommended_div[~recommended_div[['user_id', 'id']].apply(frozenset, axis=1).isin(activities[['user_id', 'content_id']].apply(frozenset, axis=1))]
recommended_div

,user_id,cluster,episode_title,id,category,title,series,episode_name,description,description2,tags,image,directors_actors,tags2,publication_date,rating,duration_sec,season,episode,features
0,0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,5.0,Ochres,326.0,Comedy,Ochres,NaN,Ochres,"Told in five parts, Ochres was inspired by the...","Told in five parts, Ochres was inspired by the...","[ABC ARTS, ARTS & CULTURE, INDIGENOUS]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW229...,[[]],"[abcarts, arts, dance, bangarra, first-nations...",2021-06-18 07:00:00,G,3417.0,NaN,,told five part ochre inspired special role och...
3,0,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1,4.0,What's For Dinner? Lunar New Year,154.0,Family,What's For Dinner? Lunar New Year,NaN,What's For Dinner? Lunar New Year,Welcome to Penny's place for Lunar New Year! I...,Welcome to Penny's place for Lunar New Year! I...,"[ABC ME, EDUCATION, FAMILY]",https://cdn.iview.abc.net.au/thumbs/i/ck/CK204...,[[]],"[abc3, childrens, primary-humanities, educatio...",2021-02-05 06:00:00,G,783.0,NaN,,welcome penny place lunar new year start race ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182,31,5.0,Australia Day Live,313.0,Comedy,Australia Day Live 2022,NaN,Australia Day Live 2022,A concert and fireworks spectacular from Sydne...,A concert and fireworks spectacular from Sydne...,"[ABC TV, ABC ARTS, ARTS & CULTURE]",https://cdn.iview.abc.net.au/thumbs/i/rv/RV210...,"[['Host Jeremy Fernandez', 'Casey Donovan', 'J...","[abc1, aussie, australia, event, concert, musi...",2022-01-26 21:31:00,PG,7871.0,NaN,,concert firework spectacular sydney featuring ...
183,31,5.0,Ochres,326.0,Comedy,Ochres,NaN,Ochres,"Told in five parts, Ochres was inspired by the...","Told in five parts, Ochres was inspired by the...","[ABC ARTS, ARTS & CULTURE, INDIGENOUS]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW229...,[[]],"[abcarts, arts, dance, bangarra, first-nations...",2021-06-18 07:00:00,G,3417.0,NaN,,told five part ochre inspired special role och...
184,32,11.0,The Broken Shore,92.0,Movies,The Broken Shore,NaN,The Broken Shore,An evocative crime thriller set against a back...,An evocative crime thriller set against a back...,"[ABC TV, ABC TV Plus, DRAMA, MOVIES, MOVIES]",https://cdn.iview.abc.net.au/thumbs/i/dr/DR121...,[['Director Rowan Woods']],"[abc1, abc2, drama, crime, film, murder, aussi...",2021-06-11 07:05:00,M,6212.0,NaN,,evocative crime thriller set background police...
185,32,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
recommended_shows_div = recommended_shows_div[~recommended_shows_div[['user_id', 'id']].apply(frozenset, axis=1).isin(activities[['user_id', 'content_id']].apply(frozenset, axis=1))]
recommended_shows_div

,user_id,cluster,index,id,title,category,description,description2,directors_actors,publication_date,rating,duration_sec,tags,tags2,image,features
1,0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,0.0,572.0,10572.0,Traces,Drama,"While attending an online forensic course, a y...",[While Kathy oversees the excavation of Marie'...,"[['Cast Molly Windsor', 'Jennifer Spence', 'La...",2021-08-15 21:27:00,M,"[2618.0, 2592.0, 2712.0, 2642.0, 2581.0, 2593.0]","[ABC TV, DRAMA]","[crime-drama, gritty, mystery, suspense, thril...",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW296...,attending online forensic course young lab ass...
4,1,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,29,0.0,572.0,10572.0,Traces,Drama,"While attending an online forensic course, a y...",[While Kathy oversees the excavation of Marie'...,"[['Cast Molly Windsor', 'Jennifer Spence', 'La...",2021-08-15 21:27:00,M,"[2618.0, 2592.0, 2712.0, 2642.0, 2581.0, 2593.0]","[ABC TV, DRAMA]","[crime-drama, gritty, mystery, suspense, thril...",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW296...,attending online forensic course young lab ass...
88,29,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
89,29,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
91,30,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
#choose 2 rows per each user_id and each cluster
recommended_films = recommended.groupby(['user_id', 'cluster']).head(2)
recommended_films

,user_id,cluster,episode_title,id,category,title,series,episode_name,description,description2,tags,image,directors_actors,tags2,publication_date,rating,duration_sec,season,episode,features
0,0,1.0,Can We Save The Reef?,152,Family,Can We Save The Reef?,NaN,Can We Save The Reef?,An epic story of Australian and international ...,An epic story of Australian and international ...,"[ABC TV, DOCUMENTARY, SCIENCE, FAMILY]",https://cdn.iview.abc.net.au/thumbs/i/sc/SC160...,[[]],"[abc1, docs, nature, science, science-week, cl...",2018-05-12 14:58:55,G,3428.0,NaN,,epic story australian international scientist ...
2,0,1.0,Sea Lions: Life By A Whisker,187,Family,Sea Lions: Life By A Whisker,NaN,Sea Lions: Life By A Whisker,Take an intimate journey inside the colony of ...,Take an intimate journey inside the colony of ...,"[ABC ME, FAMILY]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW284...,[['Narrator Sam Neill']],"[abc3, family-viewing-abcme, 3featured, family...",2022-03-16 09:45:00,G,2439.0,NaN,,take intimate journey inside colony australian...
4,0,5.0,Children Of The Revolution,112,Movies,Children Of The Revolution,NaN,Children Of The Revolution,Few knew that Stalin spent his last night in t...,Few knew that Stalin spent his last night in t...,"[ABC TV, DRAMA, MOVIES, COMEDY]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW303...,"[['Cast Judy Davis', 'Richard Roxburgh', 'Sam ...","[abc1, drama, film, comedy, australia, politic...",2021-10-01 07:00:00,M,6028.0,NaN,,knew stalin spent last night arm young austral...
5,0,5.0,Romeo & Juliet,148,Family,Romeo & Juliet,NaN,Romeo & Juliet,"Graeme Murphy's Romeo and Juliet, performed by...","Graeme Murphy's Romeo and Juliet, performed by...","[ABC TV Plus, ABC ARTS, ARTS & CULTURE, FAMILY]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW230...,[[]],"[abc2, arts, abcarts, ballet, entertainment, c...",2021-06-18 07:00:00,PG,7168.0,NaN,,graeme murphy romeo juliet performed australia...
23,0,7.0,Goldstone,106,Movies,Goldstone,NaN,Goldstone,Indigenous detective Jay Swan arrives in the t...,Indigenous detective Jay Swan arrives in the t...,"[ABC TV, ABC TV Plus, DRAMA, MOVIES, INDIGENOUS]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW109...,[['Director Ivan Sen']],"[abc1, abc2, drama, film, detective, mystery, ...",2021-08-11 07:00:00,M,6560.0,NaN,,indigenous detective jay swan arrives town gol...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
673,32,11.0,Final Rendezvous,229,Documentary,Final Rendezvous,NaN,Final Rendezvous,The collapse of a major spy case involving a f...,The collapse of a major spy case involving a f...,"[ABC TV, DOCUMENTARY]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW236...,[['Director Peter Butt']],"[abc1, docs, history, factual, australia, hist...",2021-06-18 07:00:00,PG,3372.0,NaN,,collapse major spy case involving female doubl...
674,32,11.0,NAIDOC Week Musical Celebration,306,Comedy,NAIDOC Week Musical Celebration,NaN,NAIDOC Week Musical Celebration,ABC Melbourne presents a special performance t...,ABC Melbourne presents a special performance t...,"[ABC TV Plus, ABC ARTS, ARTS & CULTURE, INDIGE...",https://cdn.iview.abc.net.au/thumbs/i/iv/IV201...,"[['Host Jacinta Parsons', 'Shelley Ware']]","[abc2, abcarts, arts, music, performance, conc...",2021-07-04 07:00:00,G,2821.0,NaN,,abc melbourne present special performance cele...
677,32,1.0,Can We Save The Reef?,152,Family,Can We Save The Reef?,NaN,Can We Save The Reef?,An epic story of Australian and international ...,An epic story of Australian and international ...,"[ABC TV, DOCUMENTARY, SCIENCE, FAMILY]",https://cdn.iview.abc.net.au/thumbs/i/sc/SC160...,[[]],"[abc1, docs, nature, science, science-week, cl...",2018-05-12 14:58:55,G,3428.0,NaN,,epic story australian international scientist ...
679,32,1.0,Sea Lions: Life By A Whisker,187,Family,Sea Lions: Life By A Whisker,NaN,Sea Lions: Life By A Whisker,Take an intimate journey inside the colony of ...,Take an intimate journey inside the colony of ..

In [35]:
recommended_shows = recommended_shows.groupby(['user_id', 'cluster']).head(2)
recommended_shows

,user_id,cluster,index,id,title,category,description,description2,directors_actors,publication_date,rating,duration_sec,tags,tags2,image,features
0,0,0.0,52.0,10052.0,Australia Remastered: Australia's Oceans,Family,Explore the incredible diversity of ocean wild...,[The Southern Ocean is the only ocean that str...,[['Host Aaron Pedersen']],2020-10-25 19:03:00,G,"[3230.0, 3195.0, 3185.0]","[ABC TV, ABC ME, DOCUMENTARY, SCIENCE, FAMILY]","[ns:be-informed, natural-world, wildlife, a:ol...",https://cdn.iview.abc.net.au/thumbs/i/do/DO184...,explore incredible diversity ocean wildlife en...
2,0,0.0,56.0,10056.0,Australia Remastered: Wild Treasures,Family,Australia's diverse and unique landscapes and ...,[The Daintree Rainforest has one of the highes...,[['Host Aaron Pedersen']],2021-01-11 21:22:47,G,"[3205.0, 3315.0, 3260.0, 3254.0, 3257.0, 3220.0]","[ABC TV, ABC ME, DOCUMENTARY, FAMILY]","[natural-world, wildlife, family-viewing-abcme...",https://cdn.iview.abc.net.au/thumbs/i/do/DO184...,australia diverse unique landscape ecosystem u...
7,0,5.0,290.0,10290.0,Little J And Big Cuz Indigenous Languages,Kids,Little J and Big Cuz are Indigenous Australian...,[Nanna is unwell so Little J and Big Cuz eager...,[[]],2021-07-31 06:00:00,G,"[760.0, 778.0, 706.0, 704.0, 720.0, 760.0, 727...","[ABC Kids, EDUCATION]","[littlejin, abc4kids, education]",https://cdn.iview.abc.net.au/thumbs/i/ck/CK194...,little j big cuz indigenous australian kid liv...
8,1,0.0,52.0,10052.0,Australia Remastered: Australia's Oceans,Family,Explore the incredible diversity of ocean wild...,[The Southern Ocean is the only ocean that str...,[['Host Aaron Pedersen']],2020-10-25 19:03:00,G,"[3230.0, 3195.0, 3185.0]","[ABC TV, ABC ME, DOCUMENTARY, SCIENCE, FAMILY]","[ns:be-informed, natural-world, wildlife, a:ol...",https://cdn.iview.abc.net.au/thumbs/i/do/DO184...,explore incredible diversity ocean wildlife en...
9,1,0.0,55.0,10055.0,Australia Remastered: Wild Australians,Family,Explore some of Australia's most iconic and fa...,"[Echidnas and platypuses are unique, the only ...",[['Host Aaron Pedersen']],2020-08-30 18:57:00,G,"[3310.0, 3330.0, 3285.0, 3185.0, 3230.0, 3294.0]","[ABC TV, ABC ME, DOCUMENTARY, FAMILY]","[ns:be-informed, natural-world, wildlife, a:ol...",https://cdn.iview.abc.net.au/thumbs/i/do/DO184...,explore australia iconic fascinating animal my...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
330,29,4.0,163.0,10163.0,Employable Me Australia,Documentary,The Rose d'Or nominated series follows six ind...,"[This uplifting, warm, and insightful series s...",[[]],2019-04-09 21:32:30,PG,"[239.0, 168.0, 3566.0, 3579.0, 235.0, 233.0, 3...","[ABC TV, DOCUMENTARY]","[real-life, endearing, informative, disability...",https://cdn.iview.abc.net.au/thumbs/i/dc/DC182...,rose dor nominated series follows six individu...
336,30,0.0,52.0,10052.0,Australia Remastered: Australia's Oceans,Family,Explore the incredible diversity of ocean wild...,[The Southern Ocean is the only ocean that str...,[['Host Aaron Pedersen']],2020-10-25 19:03:00,G,"[3230.0, 3195.0, 3185.0]","[ABC TV, ABC ME, DOCUMENTARY, SCIENCE, FAMILY]","[ns:be-informed, natural-world, wildlife, a:ol...",https://cdn.iview.abc.net.au/thumbs/i/do/DO184...,explore incredible diversity ocean wildlife en...
337,30,0.0,55.0,10055.0,Australia Remastered: Wild Australians,Family,Explore some of Australia's most iconic and fa...,"[Echidnas and platypuses are unique, the only ...",[['Host Aaron Pedersen']],2020-08-30 18:57:00,G,"[3310.0, 3330.0, 3285.0, 3185.0, 3230.0, 3294.0]","[ABC TV, ABC ME, DOCUMENTARY, FAMILY]","[ns:be-informed, natural-world, wildlife, a:ol...",https://cdn.iview.abc.net.au/thumbs/i/do/DO184...,explore australia iconic fascinating animal my...
342,30,3.0,210.0,10210.0,Griff's Great Australian Rail Trip,Documentary,British comedian and travel enthusiast Griff R...,[Journeying from Brisbane to Cairns aboard the...,[['Host Griff Rhys Jones']],2021-06-30 07:00:00,G,"[2829.0, 2785.0, 283

In [36]:
recommended_films_div = recommended_div.groupby(['user_id', 'cluster']).head(2)
recommended_films_div

,user_id,cluster,episode_title,id,category,title,series,episode_name,description,description2,tags,image,directors_actors,tags2,publication_date,rating,duration_sec,season,episode,features
0,0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,5.0,Ochres,326.0,Comedy,Ochres,NaN,Ochres,"Told in five parts, Ochres was inspired by the...","Told in five parts, Ochres was inspired by the...","[ABC ARTS, ARTS & CULTURE, INDIGENOUS]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW229...,[[]],"[abcarts, arts, dance, bangarra, first-nations...",2021-06-18 07:00:00,G,3417.0,NaN,,told five part ochre inspired special role och...
3,0,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1,4.0,What's For Dinner? Lunar New Year,154.0,Family,What's For Dinner? Lunar New Year,NaN,What's For Dinner? Lunar New Year,Welcome to Penny's place for Lunar New Year! I...,Welcome to Penny's place for Lunar New Year! I...,"[ABC ME, EDUCATION, FAMILY]",https://cdn.iview.abc.net.au/thumbs/i/ck/CK204...,[[]],"[abc3, childrens, primary-humanities, educatio...",2021-02-05 06:00:00,G,783.0,NaN,,welcome penny place lunar new year start race ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182,31,5.0,Australia Day Live,313.0,Comedy,Australia Day Live 2022,NaN,Australia Day Live 2022,A concert and fireworks spectacular from Sydne...,A concert and fireworks spectacular from Sydne...,"[ABC TV, ABC ARTS, ARTS & CULTURE]",https://cdn.iview.abc.net.au/thumbs/i/rv/RV210...,"[['Host Jeremy Fernandez', 'Casey Donovan', 'J...","[abc1, aussie, australia, event, concert, musi...",2022-01-26 21:31:00,PG,7871.0,NaN,,concert firework spectacular sydney featuring ...
183,31,5.0,Ochres,326.0,Comedy,Ochres,NaN,Ochres,"Told in five parts, Ochres was inspired by the...","Told in five parts, Ochres was inspired by the...","[ABC ARTS, ARTS & CULTURE, INDIGENOUS]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW229...,[[]],"[abcarts, arts, dance, bangarra, first-nations...",2021-06-18 07:00:00,G,3417.0,NaN,,told five part ochre inspired special role och...
184,32,11.0,The Broken Shore,92.0,Movies,The Broken Shore,NaN,The Broken Shore,An evocative crime thriller set against a back...,An evocative crime thriller set against a back...,"[ABC TV, ABC TV Plus, DRAMA, MOVIES, MOVIES]",https://cdn.iview.abc.net.au/thumbs/i/dr/DR121...,[['Director Rowan Woods']],"[abc1, abc2, drama, crime, film, murder, aussi...",2021-06-11 07:05:00,M,6212.0,NaN,,evocative crime thriller set background police...
185,32,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
recommended_shows_div = recommended_shows_div.groupby(['user_id', 'cluster']).head(2)
recommended_shows_div

,user_id,cluster,index,id,title,category,description,description2,directors_actors,publication_date,rating,duration_sec,tags,tags2,image,features
1,0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,0.0,572.0,10572.0,Traces,Drama,"While attending an online forensic course, a y...",[While Kathy oversees the excavation of Marie'...,"[['Cast Molly Windsor', 'Jennifer Spence', 'La...",2021-08-15 21:27:00,M,"[2618.0, 2592.0, 2712.0, 2642.0, 2581.0, 2593.0]","[ABC TV, DRAMA]","[crime-drama, gritty, mystery, suspense, thril...",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW296...,attending online forensic course young lab ass...
4,1,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,29,0.0,572.0,10572.0,Traces,Drama,"While attending an online forensic course, a y...",[While Kathy oversees the excavation of Marie'...,"[['Cast Molly Windsor', 'Jennifer Spence', 'La...",2021-08-15 21:27:00,M,"[2618.0, 2592.0, 2712.0, 2642.0, 2581.0, 2593.0]","[ABC TV, DRAMA]","[crime-drama, gritty, mystery, suspense, thril...",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW296...,attending online forensic course young lab ass...
88,29,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
89,29,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
91,30,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
#drop na in id column and mix recommended_films and recommended_shows
# recommended_films.dropna(subset=['id'], inplace=True)
# recommended_shows.dropna(subset=['id'], inplace=True)
#join recommended_films and recommended_shows
recommended_mix_austalia = pd.concat([recommended_films, recommended_shows])
#choose the random 6 rows per each user_id
recommended_mix_austalia = recommended_mix_austalia.groupby('user_id').apply(lambda x: x.sample(2, replace=False))
recommended_mix_austalia.reset_index(drop=True, inplace=True)


In [39]:
recommended_mix_austalia.dropna(subset=['id'], inplace=True)
recommended_mix_austalia

,user_id,cluster,episode_title,id,category,title,series,episode_name,description,description2,...,image,directors_actors,tags2,publication_date,rating,duration_sec,season,episode,features,index
0,0,7.0,Goldstone,106.0,Movies,Goldstone,NaN,Goldstone,Indigenous detective Jay Swan arrives in the t...,Indigenous detective Jay Swan arrives in the t...,...,https://cdn.iview.abc.net.au/thumbs/i/zw/ZW109...,[['Director Ivan Sen']],"[abc1, abc2, drama, film, detective, mystery, ...",2021-08-11 07:00:00,M,6560.0,NaN,,indigenous detective jay swan arrives town gol...,NaN
1,0,0.0,NaN,10052.0,Family,Australia Remastered: Australia's Oceans,NaN,NaN,Explore the incredible diversity of ocean wild...,[The Southern Ocean is the only ocean that str...,...,https://cdn.iview.abc.net.au/thumbs/i/do/DO184...,[['Host Aaron Pedersen']],"[ns:be-informed, natural-world, wildlife, a:ol...",2020-10-25 19:03:00,G,"[3230.0, 3195.0, 3185.0]",NaN,NaN,explore incredible diversity ocean wildlife en...,52.0
2,1,7.0,NaN,10059.0,Education,Australia's Prime Ministers,NaN,NaN,"This series profiles Australia's leaders, feat...","[This series profiles Australia's leaders, fea...",...,https://cdn.iview.abc.net.au/thumbs/i/zx/ZX642...,[[]],"[primary-humanities, education, abc3, gough-wh...",2022-02-28 09:00:00,G,"[183.0, 119.0, 186.0, 180.0, 199.0, 227.0, 157...",NaN,NaN,series profile australia leader featuring care...,59.0
3,1,6.0,A Few Best Men,23.0,Movies,A Few Best Men,NaN,A Few Best Men,When English lad David announces he is getting...,When English lad David announces he is getting...,...,https://cdn.iview.abc.net.au/thumbs/i/zw/ZW285...,[['Director Stephen Elliott']],"[abc1, abc2, comedy, love, relationships, roma...",2021-06-01 07:00:00,MA,5550.0,NaN,,english lad david announces getting married mi...,NaN
4,2,0.0,NaN,10052.0,Family,Australia Remastered: Australia's Oceans,NaN,NaN,Explore the incredible diversity of ocean wild...,[The Southern Ocean is the only ocean that str...,...,https://cdn.iview.abc.net.au/thumbs/i/do/DO184...,[['Host Aaron Pedersen']],"[ns:be-informed, natural-world, wildlife, a:ol...",2020-10-25 19:03:00,G,"[3230.0, 3195.0, 3185.0]",NaN,NaN,explore incredible diversity ocean wildlife en...,52.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,30,3.0,NaN,10210.0,Documentary,Griff's Great Australian Rail Trip,NaN,NaN,British comedian and travel enthusiast Griff R...,[Journeying from Brisbane to Cairns aboard the...,...,https://cdn.iview.abc.net.au/thumbs/i/zw/ZW184...,[['Host Griff Rhys Jones']],"[factual, travel, lifestyle, abc1, uk, austral...",2021-06-30 07:00:00,G,"[2829.0, 2785.0, 2831.0, 2768.0, 2821.0, 2832.0]",NaN,NaN,british comedian travel enthusiast griff rhys ...,210.0
62,31,6.0,A Few Best Men,23.0,Movies,A Few Best Men,NaN,A Few Best Men,When English lad David announces he is getting...,When English lad David announces he is getting...,...,https://cdn.iview.abc.net.au/thumbs/i/zw/ZW285...,[['Director Stephen Elliott']],"[abc1, abc2, comedy, love, relationships, roma...",2021-06-01 07:00:00,MA,5550.0,NaN,,english lad david announces getting married mi...,NaN
63,31,5.0,Romeo & Juliet,148.0,Family,Romeo & Juliet,NaN,Romeo & Juliet,"Graeme Murphy's Romeo and Juliet, performed by...","Graeme Murphy's Romeo and Juliet, performed by...",...,https://cdn.iview.abc.net.au/thumbs/i/zw/ZW230...,[[]],"[abc2, arts, abcarts, ballet, entertainment, c...",2021-06-18 07:00:00,PG,7168.0,NaN,,graeme murphy romeo juliet performed australia...,NaN
64,32,1.0,Sea Lions: Life By A Whisker,187.0,Family,Sea Lions: Life By A Whisker,NaN,Sea Lions: Life By A Whisker,Take an intimate journey inside the colony of ...,Take an intimate journey inside the colony of ...,...,https://cdn.iview.abc.net.au/thumbs/i/zw/ZW284...,[['Narrator Sam Neill']],"[abc3, family-viewing-abcme, 3featured, family...",2022-03-16 09:45:00,G,2439.0,NaN,,take intimate journey inside colony australian...,NaN


In [40]:
# recommended_films_div.dropna(subset=['id'], inplace=True)
# recommended_shows_div.dropna(subset=['id'], inplace=True)
recommended_mix_div = pd.concat([recommended_films_div, recommended_shows_div])
#choose the random 6 rows per each user_id
recommended_mix_div = recommended_mix_div.groupby('user_id').apply(lambda x: x.sample(2, replace=False))
recommended_mix_div.reset_index(drop=True, inplace=True)

In [41]:
recommended_mix_div.dropna(subset=['id'], inplace=True)

In [42]:
recommended_mix_div

,user_id,cluster,episode_title,id,category,title,series,episode_name,description,description2,...,image,directors_actors,tags2,publication_date,rating,duration_sec,season,episode,features,index
2,1,0.0,NaN,10572.0,Drama,Traces,NaN,NaN,"While attending an online forensic course, a y...",[While Kathy oversees the excavation of Marie'...,...,https://cdn.iview.abc.net.au/thumbs/i/zw/ZW296...,"[['Cast Molly Windsor', 'Jennifer Spence', 'La...","[crime-drama, gritty, mystery, suspense, thril...",2021-08-15 21:27:00,M,"[2618.0, 2592.0, 2712.0, 2642.0, 2581.0, 2593.0]",NaN,NaN,attending online forensic course young lab ass...,572.0
4,2,0.0,NaN,10572.0,Drama,Traces,NaN,NaN,"While attending an online forensic course, a y...",[While Kathy oversees the excavation of Marie'...,...,https://cdn.iview.abc.net.au/thumbs/i/zw/ZW296...,"[['Cast Molly Windsor', 'Jennifer Spence', 'La...","[crime-drama, gritty, mystery, suspense, thril...",2021-08-15 21:27:00,M,"[2618.0, 2592.0, 2712.0, 2642.0, 2581.0, 2593.0]",NaN,NaN,attending online forensic course young lab ass...,572.0
5,2,4.0,What's For Dinner? Lunar New Year,154.0,Family,What's For Dinner? Lunar New Year,NaN,What's For Dinner? Lunar New Year,Welcome to Penny's place for Lunar New Year! I...,Welcome to Penny's place for Lunar New Year! I...,...,https://cdn.iview.abc.net.au/thumbs/i/ck/CK204...,[[]],"[abc3, childrens, primary-humanities, educatio...",2021-02-05 06:00:00,G,783.0,NaN,,welcome penny place lunar new year start race ...,NaN
6,3,0.0,What Was It Like?,240.0,Documentary,What Was It Like?,NaN,What Was It Like?,Eight filmmakers with intellectual disability ...,Eight filmmakers with intellectual disability ...,...,https://cdn.iview.abc.net.au/thumbs/i/rf/RF210...,[['Director Genevieve Clay-Smith']],"[abc1, abc2, docs, diversity, cald, disability...",2021-12-20 17:01:00,PG,893.0,NaN,,eight filmmaker intellectual disability interv...,NaN
9,4,3.0,Buried,110.0,Movies,Buried,NaN,Buried,Paul Conroy is not ready to die but when he wa...,Paul Conroy is not ready to die but when he wa...,...,https://cdn.iview.abc.net.au/thumbs/i/zw/ZW288...,[['Director Rodrigo Cortes']],"[abc1, abc2, drama, mystery, thriller, film, e...",2021-06-01 07:00:00,MA,5430.0,NaN,,paul conroy ready die wake six foot undergroun...,NaN
10,5,0.0,Re-Frame,248.0,Documentary,Re-Frame 2020,NaN,Re-Frame 2020,More than 4 million Australians live with a di...,More than 4 million Australians live with a di...,...,https://cdn.iview.abc.net.au/thumbs/i/rf/RF200...,[['Host Bridie McKim']],"[abc1, disability, arts, abcarts, drama, comed...",2020-11-28 14:30:00,M,3480.0,NaN,,4 million australian live disability story tel...,NaN
12,6,3.0,Buried,110.0,Movies,Buried,NaN,Buried,Paul Conroy is not ready to die but when he wa...,Paul Conroy is not ready to die but when he wa...,...,https://cdn.iview.abc.net.au/thumbs/i/zw/ZW288...,[['Director Rodrigo Cortes']],"[abc1, abc2, drama, mystery, thriller, film, e...",2021-06-01 07:00:00,MA,5430.0,NaN,,paul conroy ready die wake six foot undergroun...,NaN
13,6,0.0,NaN,10572.0,Drama,Traces,NaN,NaN,"While attending an online forensic course, a y...",[While Kathy oversees the excavation of Marie'...,...,https://cdn.iview.abc.net.au/thumbs/i/zw/ZW296...,"[['Cast Molly Windsor', 'Jennifer Spence', 'La...","[crime-drama, gritty, mystery, suspense, thril...",2021-08-15 21:27:00,M,"[2618.0, 2592.0, 2712.0, 2642.0, 2581.0, 2593.0]",NaN,NaN,attending online forensic course young lab ass...,572.0
15,7,0.0,Re-Frame,277.0,Documentary,Re-Frame 2021,NaN,Re-Frame 2021,"Showcasing the diversity of disability, Re-Fra...","Showcasing the diversity of disability, Re-Fra...",...,https://cdn.iview.abc.net.au/thumbs/i/rf/RF210...,[[]],"[abc1, abc2, docs, diversity, cald, disability...",2021-12-03 07:00:00,M,3493.0,NaN,,showcasing diversity disability reframe brings...,NaN
16,8,0.0,What Was It Like?,240.0,Documentary,What Was It Like?,NaN,What Was It Like?,Eight filmmakers with intellectual disability ...,Eight filmmakers 